In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv('/content/drive/MyDrive/Data/Fakenews_train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.isnull().any()

id        False
title      True
author     True
text       True
label     False
dtype: bool

In [6]:
df.dropna()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [7]:
# Get the independent features
X=df.drop('label',axis=1)

In [8]:
#Get the dependent features
y=df['label']

In [9]:
y.value_counts() #checking if the dataset is balanced or not

1    10413
0    10387
Name: label, dtype: int64

In [10]:
X.shape

(20800, 4)

In [11]:
y.shape

(20800,)

In [12]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [13]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models  import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Bidirectional,Dense,Dropout

In [14]:
voc_size=5000 #vocabulary size

In [15]:
# One hot representation

In [16]:
messages=X.copy()

In [17]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [18]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
messages.reset_index(inplace=True)

In [20]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
type(messages['title'])

pandas.core.series.Series

In [23]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review=re.sub('[^a-zA-Z]',' ',str(messages['title'][i]))
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

  

In [24]:
one_hot_r=[one_hot(words,voc_size) for words in corpus]
one_hot_r

[[4525, 215, 3644, 3554, 2731, 4298, 914, 1387, 4761, 778],
 [1432, 415, 2937, 2069, 2022, 4626, 592],
 [967, 4078, 549, 1653],
 [3861, 1023, 272, 3120, 1218, 3603],
 [988, 2022, 4631, 4456, 1301, 362, 2022, 4330, 4313, 958],
 [2786,
  2817,
  1071,
  3229,
  3289,
  2348,
  4664,
  542,
  3543,
  2290,
  277,
  807,
  2468,
  869,
  592],
 [73, 73, 926, 521, 4666, 3693, 120, 4362, 4937, 3608, 2673, 1971],
 [2868, 4362, 2448, 349, 2247, 2020, 3804, 1108, 2290, 359, 1444],
 [3419, 3662, 2951, 3227, 2348, 40, 1028, 3489, 350, 1311, 2290, 359, 1444],
 [4298, 1350, 1571, 1432, 555, 3714, 2348, 2405, 2290, 359, 1444],
 [195, 4603, 2948, 2255, 1417, 3821, 721, 1481, 2348, 3901],
 [1041, 1658, 1601, 278, 3108, 4602, 4404, 4264],
 [4116, 4690, 262, 3361, 4567, 3941, 887, 4631, 3806, 53, 313],
 [3120, 4862, 2731, 3821, 2348, 555],
 [3019, 4022, 4740, 1027, 683, 1746, 1732, 3349, 1765],
 [1770, 1631, 3184, 203, 4506, 3417, 3602, 2290, 359, 1444],
 [1649, 1243, 4120, 923, 156, 2290, 359, 1444],
 

In [25]:
sent_length=20
embedded_docs=pad_sequences(one_hot_r, padding='pre',maxlen=sent_length)

In [26]:
#Creating a model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
X_final

array([[   0,    0,    0, ..., 1387, 4761,  778],
       [   0,    0,    0, ..., 2022, 4626,  592],
       [   0,    0,    0, ..., 4078,  549, 1653],
       ...,
       [   0,    0,    0, ..., 2290,  359, 1444],
       [   0,    0,    0, ..., 4145, 4878, 1249],
       [   0,    0,    0, ..., 1723,  852, 1470]], dtype=int32)

In [29]:
X_final.shape,y_final.shape

((20800, 20), (20800,))

In [30]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=0)

In [31]:
#Finally training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 10s 11ms/step - loss: 0.3086 - accuracy: 0.8604 - val_loss: 0.1929 - val_accuracy: 0.9215
Epoch 2/10
218/218 [==============================] - 2s 9ms/step - loss: 0.1342 - accuracy: 0.9474 - val_loss: 0.1835 - val_accuracy: 0.9254
Epoch 3/10
218/218 [==============================] - 2s 9ms/step - loss: 0.0887 - accuracy: 0.9670 - val_loss: 0.1972 - val_accuracy: 0.9256
Epoch 4/10
218/218 [==============================] - 2s 9ms/step - loss: 0.0553 - accuracy: 0.9812 - val_loss: 0.2472 - val_accuracy: 0.9247
Epoch 5/10
218/218 [==============================] - 2s 9ms/step - loss: 0.0325 - accuracy: 0.9896 - val_loss: 0.3357 - val_accuracy: 0.9191
Epoch 6/10
218/218 [==============================] - 2s 9ms/step - loss: 0.0194 - accuracy: 0.9936 - val_loss: 0.3860 - val_accuracy: 0.9145
Epoch 7/10
218/218 [==============================] - 2s 9ms/step - loss: 0.0138 - accuracy: 0.9957 - val_loss: 0.4276 - val_accuracy: 0.9028
Epoc

In [32]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [33]:
from sklearn.metrics import confusion_matrix

In [35]:
confusion_matrix(y_test,y_pred)

array([[3094,  296],
       [ 354, 3120]])

In [36]:
from sklearn.metrics import accuracy_score

In [38]:
accuracy_score(y_test,y_pred)

0.9053030303030303

In [39]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [41]:
confusion_matrix(y_test,y_pred)

array([[3094,  296],
       [ 354, 3120]])

In [42]:
accuracy_score(y_test,y_pred)

0.9053030303030303